In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import os
from main.utils.data_manage_utils import read_table_from_subfolder
# Variables
ROOT_PATH = os.path.abspath("../../")
PREP_FOLDER = os.path.join(ROOT_PATH, "data/preparation/prepped_files")

In [2]:
atl_flights_df = pd.read_pickle(os.path.join(PREP_FOLDER, "01_flights.pkl"))
atl_runways_df = pd.read_pickle(os.path.join(PREP_FOLDER, "03_runways.pkl"))
aircraft_df = pd.read_pickle(os.path.join(PREP_FOLDER, "04_a_aircrafts.pkl"))
aircraft_numbers_df = pd.read_pickle(os.path.join(PREP_FOLDER, "04_b_aircrafts.pkl"))
atl_metar_df = pd.read_pickle(os.path.join(PREP_FOLDER, "05_metar.pkl"))
atl_notam_df = pd.read_pickle(os.path.join(PREP_FOLDER, "06_notam.pkl"))

#Merging flight data with aircraft data
atl_flights_df = atl_flights_df.join(aircraft_numbers_df[["Aircraft", "TAIL_NUM"]].set_index("TAIL_NUM"), on="TAIL_NUM")
atl_flights_df

,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_DATE,CRS_ARR_DATE,ACT_DEP_DATE,ACT_ARR_DATE,PREV_FLIGHTS_1H,ARR_DELAY,Aircraft
12864,NaN,N353NW,NaN,-8,170.0,1199.0,2016-01-01 00:15:00,2016-01-01 05:05:00,2016-01-01 00:07:00,2016-01-01 04:33:00,0,-32,Airbus A320
12377,NaN,N849DN,NaN,-3,209.0,1590.0,2016-01-01 00:55:00,2016-01-01 06:24:00,2016-01-01 00:52:00,2016-01-01 06:18:00,0,-6,Boeing 737 NG / Max
8481,NaN,N201FR,NaN,4,225.0,1747.0,2016-01-01 00:20:00,2016-01-01 07:05:00,2016-01-01 00:24:00,2016-01-01 06:55:00,1,-10,Airbus A320
2658,NaN,N8657B,NaN,-5,105.0,515.0,2016-01-01 05:40:00,2016-01-01 07:25:00,2016-01-01 05:35:00,2016-01-01 07:12:00,1,-13,Boeing 737 NG / Max
835,NaN,N210WN,NaN,-3,115.0,581.0,2016-01-01 05:30:00,2016-01-01 07:25:00,2016-01-01 05:27:00,2016-01-01 07:09:00,1,-16,Boeing 737 NG / Max
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11292052,-11.0,N391DA,-6.0,-2,264.0,2092.0,2017-12-31 23:25:00,2018-01-01 06:49:00,2017-12-31 23:23:00,2018-01-01 06:23:00,2,-26,Boeing 737 NG / Max
11285768,-5.0,N3CDAA,-5.0,-2,257.0,1947.0,2017-12-31 23:47:00,2018-01-01 07:04:00,2017-12-31 23:45:00,2018-01-01 06:38:00,2,-26,NaN
11290999,-11.0,N804DN,-5.0,-1,246.0,1947.0,2017-12-31 23:59:00,2018-01-01 07:05:00,2017-12-31 23:58:00,2018-01-01 07:05:00,3,0,Boeing 737 NG / Max
11290527,-11.0,N862DN,-11.0,-4,276.0,2139.0,2017-12-31 23:30:00,2018-01-01 07:06:00,2017-12-31 23:26:00,2018-01-01 06:33:00,4,-33,Boeing 737 NG / Max


In [3]:
print("Number of null values in aircraft column: " + str(atl_flights_df["Aircraft"].isna().sum()) + " of " + str(len(atl_flights_df)) + " = {:0.3f}%".format(atl_flights_df["Aircraft"].isna().sum()/len(atl_flights_df)))

Number of null values in aircraft column: 16864 of 739429 = 0.023%


In [4]:
#Dropping aircrafts with null values
atl_flights_df = atl_flights_df[atl_flights_df["Aircraft"].notna()]

In [5]:
aircraft_df = aircraft_df.rename(columns={"Model":"Aircraft"})
atl_flights_df = atl_flights_df.join(aircraft_df.set_index("Aircraft"), on="Aircraft").drop(["TAIL_NUM","ICAO Code","Aircraft"],axis=1)
atl_flights_df

,OP_UNIQUE_CARRIER,ORIGIN,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_DATE,CRS_ARR_DATE,ACT_DEP_DATE,ACT_ARR_DATE,PREV_FLIGHTS_1H,ARR_DELAY,Physical Class (Engine),Engines,Approach Speed(Vref),Wingtip Configuration,"Wingspan, ft","Length, ft","Tail Height, ft(@ OEW)",MTOW,"Parking Area (WS x Length), sf"
12864,NaN,NaN,-8,170.0,1199.0,2016-01-01 00:15:00,2016-01-01 05:05:00,2016-01-01 00:07:00,2016-01-01 04:33:00,0,-32,Jet,2,136,wingtip fences,111.88,123.27,39.63,171961,13791
12377,NaN,NaN,-3,209.0,1590.0,2016-01-01 00:55:00,2016-01-01 06:24:00,2016-01-01 00:52:00,2016-01-01 06:18:00,0,-6,Jet,2,141,no winglets,112.58,138.17,41.42,174200,15555
8481,NaN,NaN,4,225.0,1747.0,2016-01-01 00:20:00,2016-01-01 07:05:00,2016-01-01 00:24:00,2016-01-01 06:55:00,1,-10,Jet,2,136,wingtip fences,111.88,123.27,39.63,171961,13791
2658,NaN,NaN,-5,105.0,515.0,2016-01-01 05:40:00,2016-01-01 07:25:00,2016-01-01 05:35:00,2016-01-01 07:12:00,1,-13,Jet,2,141,no winglets,112.58,138.17,41.42,174200,15555
835,NaN,NaN,-3,115.0,581.0,2016-01-01 05:30:00,2016-01-01 07:25:00,2016-01-01 05:27:00,2016-01-01 07:09:00,1,-16,Jet,2,141,no winglets,112.58,138.17,41.42,174200,15555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11291377,-11.0,-9.0,-11,220.0,1747.0,2017-12-31 23:59:00,2018-01-01 06:39:00,2017-12-31 23:48:00,2018-01-01 06:19:00,2,-20,Jet,2,141,no winglets,112.58,138.17,41.42,174200,15555
11292052,-11.0,-6.0,-2,264.0,2092.0,2017-12-31 23:25:00,2018-01-01 06:49:00,2017-12-31 23:23:00,2018-01-01 06:23:00,2,-26,Jet,2,141,no winglets,112.58,138.17,41.42,174200,15555
11290999,-11.0,-5.0,-1,246.0,1947.0,2017-12-31 23:59:00,2018-01-01 07:05:00,2017-12-31 23:58:00,2018-01-01 07:05:00,3,0,Jet,2,141,no winglets,112.58,138.17,41.42,174200,15555
11290527,-11.0,-11.0,-4,276.0,2139.0,2017-12-31 23:30:00,2018-01-01 07:06:00,2017-12-31 23:26:00,2018-01-01 06:33:00,4,-33,Jet,2,141,no winglets,112.58,138.17,41.42,174200,15555


In [6]:
atl_metar_df = atl_metar_df.reset_index()
atl_metar_df["date"] = pd.to_datetime(atl_metar_df["date"] + " " + atl_metar_df["time"], format="%Y-%m-%d %H")
atl_metar_df = atl_metar_df.drop(["time","wxcodes"], axis="columns")
atl_metar_df

,date,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,...,EVENT_FG,EVENT_FU,EVENT_GR,EVENT_GS,EVENT_HZ,EVENT_IC,EVENT_RA,EVENT_SG,EVENT_SN,EVENT_TS
0,2016-01-01 00:00:00,51.98,42.08,68.86,320.0,10.0,0.0,30.19,1022.5,10.0,...,0,0,0,0,0,0,0,0,0,0
1,2016-01-01 01:00:00,50.00,41.00,71.07,330.0,7.0,0.0,30.18,1022.2,10.0,...,0,0,0,0,0,0,0,0,0,0
2,2016-01-01 02:00:00,48.92,41.00,73.99,310.0,12.0,0.0,30.18,1022.0,10.0,...,0,0,0,0,0,0,0,0,0,0
3,2016-01-01 03:00:00,46.94,39.92,76.43,320.0,13.0,0.0,30.17,1021.9,10.0,...,0,0,0,0,0,0,0,0,0,0
4,2016-01-01 04:00:00,46.04,39.02,76.35,320.0,10.0,0.0,30.18,1022.0,10.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2017-12-30 19:00:00,50.00,32.00,49.81,290.0,12.0,0.0,30.07,1018.6,10.0,...,0,0,0,0,0,0,0,0,0,0
17516,2017-12-30 20:00:00,51.10,32.00,47.81,300.0,11.0,0.0,30.06,1018.6,10.0,...,0,0,0,0,0,0,0,0,0,0
17517,2017-12-30 21:00:00,48.00,30.90,51.34,300.0,11.0,0.0,30.07,1018.9,10.0,...,0,0,0,0,0,0,0,0,0,0
17518,2017-12-30 22:00:00,46.00,30.90,55.36,290.0,9.0,0.0,30.07,1018.9,10.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
atl_flights_df = atl_flights_df[atl_flights_df["CRS_DEP_DATE"].apply(lambda x: x.year == 2017)]
atl_metar_df = atl_metar_df[atl_metar_df["date"].apply(lambda x: x.year == 2017)]
atl_metar_df

,date,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,...,EVENT_FG,EVENT_FU,EVENT_GR,EVENT_GS,EVENT_HZ,EVENT_IC,EVENT_RA,EVENT_SG,EVENT_SN,EVENT_TS
8784,2017-01-01 00:00:00,43.00,36.00,76.120,140.0,6.0,0.00,30.100,1020.2,10.0,...,0,0,0,0,0,0,0,0,0,0
8785,2017-01-01 01:00:00,42.10,36.00,78.800,150.0,6.0,0.01,30.110,1020.1,10.0,...,0,0,0,0,0,0,1,0,0,0
8786,2017-01-01 02:00:00,41.00,37.00,85.540,140.0,6.0,0.02,30.110,1020.0,9.0,...,0,0,0,0,0,0,1,0,0,0
8787,2017-01-01 03:00:00,42.10,37.90,84.930,150.0,8.0,0.02,30.110,1020.0,10.0,...,0,0,0,0,0,0,1,0,0,0
8788,2017-01-01 04:00:00,41.55,39.45,92.245,150.0,7.0,0.02,30.105,1019.8,9.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2017-12-30 19:00:00,50.00,32.00,49.810,290.0,12.0,0.00,30.070,1018.6,10.0,...,0,0,0,0,0,0,0,0,0,0
17516,2017-12-30 20:00:00,51.10,32.00,47.810,300.0,11.0,0.00,30.060,1018.6,10.0,...,0,0,0,0,0,0,0,0,0,0
17517,2017-12-30 21:00:00,48.00,30.90,51.340,300.0,11.0,0.00,30.070,1018.9,10.0,...,0,0,0,0,0,0,0,0,0,0
17518,2017-12-30 22:00:00,46.00,30.90,55.360,290.0,9.0,0.00,30.070,1018.9,10.0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
all_flight_dates = atl_flights_df["CRS_DEP_DATE"].to_numpy()
all_metar_dates = atl_metar_df["date"].to_numpy()
indexes_of_min_deltas = [np.argmin(np.absolute(all_metar_dates - date)) for date in all_flight_dates]

In [9]:
closest_date = all_metar_dates[indexes_of_min_deltas]
closest_date

array(['2017-01-01T01:00:00.000000000', '2017-01-01T05:00:00.000000000',
       '2017-01-01T06:00:00.000000000', ...,
       '2017-12-30T23:00:00.000000000', '2017-12-30T23:00:00.000000000',
       '2017-12-30T23:00:00.000000000'], dtype='datetime64[ns]')

In [10]:
atl_flights_df["MERGE_DATES"] = closest_date
atl_flights_df = atl_flights_df.join(atl_metar_df.set_index("date"), on="MERGE_DATES").drop(["MERGE_DATES"], axis="columns")
atl_flights_df

C:\Users\chrig\AppData\Local\Temp\ipykernel_13896\125089621.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atl_flights_df["MERGE_DATES"] = closest_date


,OP_UNIQUE_CARRIER,ORIGIN,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_DATE,CRS_ARR_DATE,ACT_DEP_DATE,ACT_ARR_DATE,PREV_FLIGHTS_1H,...,EVENT_FG,EVENT_FU,EVENT_GR,EVENT_GS,EVENT_HZ,EVENT_IC,EVENT_RA,EVENT_SG,EVENT_SN,EVENT_TS
5626534,-11.0,-8.0,-2,223.0,1590.0,2017-01-01 00:55:00,2017-01-01 06:38:00,2017-01-01 00:53:00,2017-01-01 06:31:00,4,...,0,0,0,0,0,0,1,0,0,0
5627321,-8.0,-8.0,-4,88.0,404.0,2017-01-01 05:15:00,2017-01-01 06:43:00,2017-01-01 05:11:00,2017-01-01 06:32:00,5,...,0,0,0,0,0,0,1,0,0,0
5622325,-8.0,-7.0,302,57.0,106.0,2017-01-01 05:49:00,2017-01-01 06:46:00,2017-01-01 10:51:00,2017-01-01 12:19:00,7,...,0,0,0,0,0,0,1,0,0,0
5626430,-11.0,-9.0,-2,98.0,447.0,2017-01-01 05:30:00,2017-01-01 07:08:00,2017-01-01 05:28:00,2017-01-01 07:19:00,6,...,0,0,0,0,0,0,1,0,0,0
5625623,-11.0,-9.0,-4,76.0,226.0,2017-01-01 06:00:00,2017-01-01 07:16:00,2017-01-01 05:56:00,2017-01-01 07:08:00,7,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11291377,-11.0,-9.0,-11,220.0,1747.0,2017-12-31 23:59:00,2018-01-01 06:39:00,2017-12-31 23:48:00,2018-01-01 06:19:00,2,...,0,0,0,0,0,0,0,0,0,0
11292052,-11.0,-6.0,-2,264.0,2092.0,2017-12-31 23:25:00,2018-01-01 06:49:00,2017-12-31 23:23:00,2018-01-01 06:23:00,2,...,0,0,0,0,0,0,0,0,0,0
11290999,-11.0,-5.0,-1,246.0,1947.0,2017-12-31 23:59:00,2018-01-01 07:05:00,2017-12-31 23:58:00,2018-01-01 07:05:00,3,...,0,0,0,0,0,0,0,0,0,0
11290527,-11.0,-11.0,-4,276.0,2139.0,2017-12-31 23:30:00,2018-01-01 07:06:00,2017-12-31 23:26:00,2018-01-01 06:33:00,4,...,0,0,0,0,0,0,0,0,0,0


In [11]:
atl_notam_df["NOTAM_Effective_Date"] = pd.to_datetime(atl_notam_df["NOTAM_Effective_Date"] + " " + atl_notam_df["NOTAM_Effective_Time"], format="%m/%d/%Y %H")
atl_notam_df["NOTAM_Expiration_Date"] = atl_notam_df["NOTAM_Effective_Date"].apply(lambda x: x + timedelta(days=1))
atl_notam_df

,index,NOTAM_Effective_Date,NOTAM_Effective_Time,NOTAM_Expiration_Date,NOTAM_Expiration_Time,08L/26R,08R/26L,09L/27R,09R/27L,10/28
0,28,2017-01-01 17:00:00,17,2017-01-02 17:00:00,17,0.0,0.0,0.0,1.0,0.0
1,29,2017-01-01 17:00:00,17,2017-01-02 17:00:00,17,0.0,0.0,1.0,0.0,0.0
2,30,2017-01-01 17:00:00,17,2017-01-02 17:00:00,17,0.0,1.0,0.0,0.0,0.0
3,31,2017-01-01 17:00:00,17,2017-01-02 17:00:00,17,1.0,0.0,0.0,0.0,0.0
4,32,2017-01-01 17:00:00,17,2017-01-02 17:00:00,17,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
3563,16115,2017-12-27 06:00:00,06,2017-12-28 06:00:00,08,3.0,0.0,0.0,0.0,0.0
3564,16127,2017-12-27 03:00:00,03,2017-12-28 03:00:00,23,2.0,0.0,0.0,0.0,0.0
3565,16128,2017-12-27 03:00:00,03,2017-12-28 03:00:00,23,0.0,2.0,0.0,0.0,0.0
3566,16130,2017-12-27 03:00:00,03,2017-12-28 03:00:00,23,2.0,0.0,0.0,0.0,0.0


In [12]:
atl_notam_df = atl_notam_df.drop(["index", "NOTAM_Expiration_Time", "NOTAM_Effective_Time"],axis="columns")
atl_notam_df

,NOTAM_Effective_Date,NOTAM_Expiration_Date,08L/26R,08R/26L,09L/27R,09R/27L,10/28
0,2017-01-01 17:00:00,2017-01-02 17:00:00,0.0,0.0,0.0,1.0,0.0
1,2017-01-01 17:00:00,2017-01-02 17:00:00,0.0,0.0,1.0,0.0,0.0
2,2017-01-01 17:00:00,2017-01-02 17:00:00,0.0,1.0,0.0,0.0,0.0
3,2017-01-01 17:00:00,2017-01-02 17:00:00,1.0,0.0,0.0,0.0,0.0
4,2017-01-01 17:00:00,2017-01-02 17:00:00,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
3563,2017-12-27 06:00:00,2017-12-28 06:00:00,3.0,0.0,0.0,0.0,0.0
3564,2017-12-27 03:00:00,2017-12-28 03:00:00,2.0,0.0,0.0,0.0,0.0
3565,2017-12-27 03:00:00,2017-12-28 03:00:00,0.0,2.0,0.0,0.0,0.0
3566,2017-12-27 03:00:00,2017-12-28 03:00:00,2.0,0.0,0.0,0.0,0.0


In [13]:
atl_notam_df = atl_notam_df.groupby(["NOTAM_Effective_Date","NOTAM_Expiration_Date"]).max()
atl_notam_df = atl_notam_df.reset_index()
atl_notam_df

,NOTAM_Effective_Date,NOTAM_Expiration_Date,08L/26R,08R/26L,09L/27R,09R/27L,10/28
0,2016-12-31 18:00:00,2017-01-01 18:00:00,1.0,1.0,1.0,1.0,1.0
1,2017-01-01 17:00:00,2017-01-02 17:00:00,1.0,1.0,1.0,1.0,1.0
2,2017-01-02 20:00:00,2017-01-03 20:00:00,1.0,1.0,1.0,1.0,1.0
3,2017-01-03 06:00:00,2017-01-04 06:00:00,0.0,0.0,0.0,3.0,0.0
4,2017-01-03 14:00:00,2017-01-04 14:00:00,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...
1100,2017-12-23 23:00:00,2017-12-24 23:00:00,2.0,0.0,0.0,0.0,0.0
1101,2017-12-25 17:00:00,2017-12-26 17:00:00,2.0,0.0,0.0,0.0,0.0
1102,2017-12-27 03:00:00,2017-12-28 03:00:00,2.0,2.0,0.0,0.0,0.0
1103,2017-12-27 06:00:00,2017-12-28 06:00:00,3.0,0.0,0.0,0.0,0.0


In [14]:
all_notam_effective_dates = atl_notam_df["NOTAM_Effective_Date"].to_numpy()
all_notam_expiration_dates = atl_notam_df["NOTAM_Expiration_Date"].to_numpy()
all_flight_dates

array(['2017-01-01T00:55:00.000000000', '2017-01-01T05:15:00.000000000',
       '2017-01-01T05:49:00.000000000', ...,
       '2017-12-31T23:59:00.000000000', '2017-12-31T23:30:00.000000000',
       '2017-12-31T17:55:00.000000000'], dtype='datetime64[ns]')

In [15]:
all_notam_effective_dates

array(['2016-12-31T18:00:00.000000000', '2017-01-01T17:00:00.000000000',
       '2017-01-02T20:00:00.000000000', ...,
       '2017-12-27T03:00:00.000000000', '2017-12-27T06:00:00.000000000',
       '2017-12-27T07:00:00.000000000'], dtype='datetime64[ns]')

In [16]:
is_above_effective = [ date >= all_notam_effective_dates for date in all_flight_dates]
is_beneath_expiration =  [date < all_notam_expiration_dates for date in all_flight_dates]
flight_in_notam_matrix = np.logical_and(is_above_effective, is_beneath_expiration)

In [17]:
flight_in_notam_matrix

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [18]:
arr = []
for idx, flight in enumerate(flight_in_notam_matrix):
    for i in np.nonzero(flight)[0]:
        arr.append([idx, all_flight_dates[idx],all_notam_effective_dates[i]])
flight_notam_df = pd.DataFrame(arr, columns=["ID","flight_date","notam_date"])
flight_notam_df

,ID,flight_date,notam_date
0,0,2017-01-01 00:55:00,2016-12-31 18:00:00
1,1,2017-01-01 05:15:00,2016-12-31 18:00:00
2,2,2017-01-01 05:49:00,2016-12-31 18:00:00
3,3,2017-01-01 05:30:00,2016-12-31 18:00:00
4,4,2017-01-01 06:00:00,2016-12-31 18:00:00
...,...,...,...
1064842,345133,2017-12-28 06:15:00,2017-12-27 07:00:00
1064843,345139,2017-12-28 06:15:00,2017-12-27 07:00:00
1064844,345142,2017-12-28 06:45:00,2017-12-27 07:00:00
1064845,345143,2017-12-28 06:30:00,2017-12-27 07:00:00


In [19]:
flight_notam_df = flight_notam_df.join(atl_notam_df.set_index("NOTAM_Effective_Date"), on="notam_date").drop(["notam_date","ID"],axis="columns")
flight_notam_df = flight_notam_df.groupby(["flight_date"]).max()
flight_notam_df = flight_notam_df.reset_index()
flight_notam_df = flight_notam_df.drop(["NOTAM_Expiration_Date"], axis=1)
flight_notam_df

,flight_date,08L/26R,08R/26L,09L/27R,09R/27L,10/28
0,2017-01-01 00:55:00,1.0,1.0,1.0,1.0,1.0
1,2017-01-01 05:15:00,1.0,1.0,1.0,1.0,1.0
2,2017-01-01 05:20:00,1.0,1.0,1.0,1.0,1.0
3,2017-01-01 05:30:00,1.0,1.0,1.0,1.0,1.0
4,2017-01-01 05:35:00,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
124155,2017-12-28 06:39:00,1.0,1.0,1.0,1.0,1.0
124156,2017-12-28 06:40:00,1.0,1.0,1.0,1.0,1.0
124157,2017-12-28 06:45:00,1.0,1.0,1.0,1.0,1.0
124158,2017-12-28 06:55:00,1.0,1.0,1.0,1.0,1.0


In [20]:
atl_flights_df = atl_flights_df.join(flight_notam_df.set_index("flight_date"), on="CRS_DEP_DATE")
atl_flights_df

,OP_UNIQUE_CARRIER,ORIGIN,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_DATE,CRS_ARR_DATE,ACT_DEP_DATE,ACT_ARR_DATE,PREV_FLIGHTS_1H,...,EVENT_IC,EVENT_RA,EVENT_SG,EVENT_SN,EVENT_TS,08L/26R,08R/26L,09L/27R,09R/27L,10/28
5626534,-11.0,-8.0,-2,223.0,1590.0,2017-01-01 00:55:00,2017-01-01 06:38:00,2017-01-01 00:53:00,2017-01-01 06:31:00,4,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
5627321,-8.0,-8.0,-4,88.0,404.0,2017-01-01 05:15:00,2017-01-01 06:43:00,2017-01-01 05:11:00,2017-01-01 06:32:00,5,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
5622325,-8.0,-7.0,302,57.0,106.0,2017-01-01 05:49:00,2017-01-01 06:46:00,2017-01-01 10:51:00,2017-01-01 12:19:00,7,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
5626430,-11.0,-9.0,-2,98.0,447.0,2017-01-01 05:30:00,2017-01-01 07:08:00,2017-01-01 05:28:00,2017-01-01 07:19:00,6,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
5625623,-11.0,-9.0,-4,76.0,226.0,2017-01-01 06:00:00,2017-01-01 07:16:00,2017-01-01 05:56:00,2017-01-01 07:08:00,7,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11291377,-11.0,-9.0,-11,220.0,1747.0,2017-12-31 23:59:00,2018-01-01 06:39:00,2017-12-31 23:48:00,2018-01-01 06:19:00,2,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
11292052,-11.0,-6.0,-2,264.0,2092.0,2017-12-31 23:25:00,2018-01-01 06:49:00,2017-12-31 23:23:00,2018-01-01 06:23:00,2,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
11290999,-11.0,-5.0,-1,246.0,1947.0,2017-12-31 23:59:00,2018-01-01 07:05:00,2017-12-31 23:58:00,2018-01-01 07:05:00,3,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
11290527,-11.0,-11.0,-4,276.0,2139.0,2017-12-31 23:30:00,2018-01-01 07:06:00,2017-12-31 23:26:00,2018-01-01 06:33:00,4,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [21]:
atl_flights_df[["08L/26R","08R/26L","09L/27R","09R/27L","10/28"]] = atl_flights_df[["08L/26R","08R/26L","09L/27R","09R/27L","10/28"]].fillna(0)

In [46]:
def get_feature_tex_table(df : pd.DataFrame):
    tex_arr = []
    for col in df.columns:
        range = f"[{df[col].min()};{df[col].max()}]"
        tex_arr.append([col,range,df[col].iloc[0]])
    tex_df = pd.DataFrame(data=tex_arr, index=None, columns=["Name", "Range", "Example"])
    s = tex_df.style
    s.caption = None
    s.format({
        ("Numeric", "Floats"): '{:.2f}'
    })
    s.format(escape="latex")
    s.hide(level=0, axis=0)
    return s.to_latex( hrules=True) 

print(get_feature_tex_table(df = atl_flights_df))

TypeError: to_latex() got an unexpected keyword argument 'hrule'

In [22]:
atl_flights_df

,OP_UNIQUE_CARRIER,ORIGIN,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_DATE,CRS_ARR_DATE,ACT_DEP_DATE,ACT_ARR_DATE,PREV_FLIGHTS_1H,...,EVENT_IC,EVENT_RA,EVENT_SG,EVENT_SN,EVENT_TS,08L/26R,08R/26L,09L/27R,09R/27L,10/28
5626534,-11.0,-8.0,-2,223.0,1590.0,2017-01-01 00:55:00,2017-01-01 06:38:00,2017-01-01 00:53:00,2017-01-01 06:31:00,4,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
5627321,-8.0,-8.0,-4,88.0,404.0,2017-01-01 05:15:00,2017-01-01 06:43:00,2017-01-01 05:11:00,2017-01-01 06:32:00,5,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
5622325,-8.0,-7.0,302,57.0,106.0,2017-01-01 05:49:00,2017-01-01 06:46:00,2017-01-01 10:51:00,2017-01-01 12:19:00,7,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
5626430,-11.0,-9.0,-2,98.0,447.0,2017-01-01 05:30:00,2017-01-01 07:08:00,2017-01-01 05:28:00,2017-01-01 07:19:00,6,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
5625623,-11.0,-9.0,-4,76.0,226.0,2017-01-01 06:00:00,2017-01-01 07:16:00,2017-01-01 05:56:00,2017-01-01 07:08:00,7,...,0,1,0,0,0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11291377,-11.0,-9.0,-11,220.0,1747.0,2017-12-31 23:59:00,2018-01-01 06:39:00,2017-12-31 23:48:00,2018-01-01 06:19:00,2,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
11292052,-11.0,-6.0,-2,264.0,2092.0,2017-12-31 23:25:00,2018-01-01 06:49:00,2017-12-31 23:23:00,2018-01-01 06:23:00,2,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
11290999,-11.0,-5.0,-1,246.0,1947.0,2017-12-31 23:59:00,2018-01-01 07:05:00,2017-12-31 23:58:00,2018-01-01 07:05:00,3,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
11290527,-11.0,-11.0,-4,276.0,2139.0,2017-12-31 23:30:00,2018-01-01 07:06:00,2017-12-31 23:26:00,2018-01-01 06:33:00,4,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [23]:
atl_flights_df.to_pickle(os.path.join(PREP_FOLDER, "07_integration.pkl"))